In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, collect_list, to_timestamp
from pyspark.sql.types import LongType

In [ ]:
# Инициализация SparkSession
spark = SparkSession.builder.appName("SumOfOdds").getOrCreate()

In [ ]:
# Определение UDF для суммирования нечётных чисел
def sum_odds(numbers):
    return sum(x for x in numbers if x % 2 != 0)

In [ ]:
# Регистрация UDF
sum_odds_udf = udf(sum_odds, LongType())

In [ ]:
# Чтение потока данных с использованием источника "rate"
df = spark.readStream.format("rate").load()

In [ ]:
# Добавление метки времени
df = df.withColumn("timestamp", to_timestamp("timestamp"))

In [ ]:
# Группировка данных и агрегация с использованием collect_list и UDF
df_with_sum_odds = df.groupBy().agg(sum_odds_udf(collect_list("value")).alias("sum_odds"))

In [ ]:
# Вывод результата с режимом вывода 'update'
query = df_with_sum_odds.writeStream.outputMode("update").format("console").start()
query.awaitTermination()